In [1]:
# ============================================================================
# COMPLETE INSPECTION CODE FOR skillsnetwork.prepare()
# ============================================================================

import skillsnetwork
import inspect
import os

# Step 1: Get the prepare function source code
print("=== INSPECTING PREPARE METHOD ===")
try:
    prepare_source = inspect.getsource(skillsnetwork.prepare)
    print("PREPARE SOURCE CODE:")
    print(prepare_source)
    print("\n" + "="*60)
    
    # Get the function signature
    sig = inspect.signature(skillsnetwork.prepare)
    print(f"FUNCTION SIGNATURE: prepare{sig}")
    print("\n" + "="*60)
    
    # Get docstring
    doc = skillsnetwork.prepare.__doc__
    print("DOCUMENTATION:")
    print(doc)
    
except Exception as e:
    print(f"Could not get prepare source: {e}")

# Step 2: Check for any helper functions that prepare() might use
print("\n=== CHECKING FOR HELPER FUNCTIONS ===")

# Look in the core.py file for any prepare-related functions
core_file = os.path.join(os.path.dirname(skillsnetwork.__file__), 'core.py')

with open(core_file, 'r') as f:
    content = f.read()

# Search for prepare-related code
lines = content.split('\n')
for i, line in enumerate(lines):
    if 'def prepare' in line or 'def _prepare' in line or 'prepare(' in line:
        print(f"Found prepare-related code at line {i+1}:")
        # Show context around the line
        start = max(0, i-3)
        end = min(len(lines), i+20)
        for j in range(start, end):
            marker = ">>> " if j == i else "    "
            print(f"{marker}{j+1:3}: {lines[j]}")
        print("\n" + "-"*40)

# Step 3: Check for any extraction/archive handling functions
print("\n=== CHECKING FOR ARCHIVE FUNCTIONS ===")
for i, line in enumerate(lines):
    if any(keyword in line.lower() for keyword in ['tar', 'zip', 'extract', 'unpack']):
        if 'def ' in line or 'import ' in line:
            print(f"Line {i+1}: {line.strip()}")

# Step 4: Look for prepare_dataset function too
print("\n=== CHECKING prepare_dataset ===")
try:
    if hasattr(skillsnetwork, 'prepare_dataset'):
        prepare_dataset_source = inspect.getsource(skillsnetwork.prepare_dataset)
        print("PREPARE_DATASET SOURCE:")
        print(prepare_dataset_source)
except Exception as e:
    print(f"prepare_dataset inspection failed: {e}")

=== INSPECTING PREPARE METHOD ===
PREPARE SOURCE CODE:
async def prepare(
    url: str, path: Optional[str] = None, verbose: bool = True, overwrite: bool = False
) -> None:
    """
    Prepares a dataset for learners. Downloads a dataset from the given url,
    decompresses it if necessary. If not using jupyterlite, will extract to
    /tmp and and symlink it so it's available at the desired path.

    >>> import skillsnetwork
    >>> await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/images.tar.gz")
    Saved to '.'

    :param url: The URL to download the dataset from.
    :param path: The path the dataset will be available at. Current working directory by default.
    :param verbose=True: Prints saved path if True.
    :param overwrite=False: If True, overwrites any existing files at destination.
    :raise InvalidURLException: When URL is invalid.
    :raise FileExistsError: When a file t

In [2]:
# ============================================================================
# GET THE MISSING HELPER FUNCTIONS FOR prepare()
# ============================================================================

import skillsnetwork
import inspect
import os

# The prepare() function uses these helper functions we need to find:
missing_functions = ['_verify_files_dont_exist', '_is_file_to_symlink']

print("=== SEARCHING FOR MISSING HELPER FUNCTIONS ===")

# Read the entire core.py file and search for these functions
core_file = os.path.join(os.path.dirname(skillsnetwork.__file__), 'core.py')

with open(core_file, 'r') as f:
    content = f.read()

lines = content.split('\n')

for func_name in missing_functions:
    print(f"\n{'='*20} {func_name} {'='*20}")
    found = False
    
    for i, line in enumerate(lines):
        if f'def {func_name}' in line:
            found = True
            print(f"Found at line {i+1}:")
            
            # Get the complete function (until next function or end)
            start = i
            indent_level = len(line) - len(line.lstrip())
            
            # Find the end of this function
            end = len(lines)
            for j in range(i+1, len(lines)):
                current_line = lines[j]
                if current_line.strip() and len(current_line) - len(current_line.lstrip()) <= indent_level:
                    if current_line.strip().startswith('def ') or current_line.strip().startswith('class '):
                        end = j
                        break
            
            # Print the complete function
            for k in range(start, end):
                print(f"{k+1:3}: {lines[k]}")
            break
    
    if not found:
        print(f"❌ {func_name} not found in core.py")

# Also check what imports are needed at the top
print(f"\n{'='*20} IMPORTS NEEDED {'='*20}")
for i, line in enumerate(lines[:30]):  # Check first 30 lines for imports
    if line.strip().startswith('import ') or line.strip().startswith('from '):
        print(f"{i+1:3}: {line}")

=== SEARCHING FOR MISSING HELPER FUNCTIONS ===

==================== _verify_files_dont_exist ====================
Found at line 124:
124: def _verify_files_dont_exist(
125:     paths: Iterable[Path], remove_if_exist: bool = False
126: ) -> None:
127:     """
128:     Verifies all paths in 'paths' don't exist.
129:     :param paths: A iterable of pathlib.Path s.
130:     :param remove_if_exist=False: Remove each file at each path in paths if they already exist.
131:     :returns: None
132:     :raises FileExistsError: On the first path found that already exists if remove_if_exist is False.
133:     """
134:     for path in paths:
135:         # Could be a broken symlink => path.exists() is False
136:         if path.exists() or path.is_symlink():
137:             if remove_if_exist:
138:                 while path.is_symlink():
139:                     temp = Path(os.readlink(path))
140:                     path.unlink()
141:                     path = temp
142:                 if path

In [4]:
# Find the _rmrf function
import skillsnetwork
import os

core_file = os.path.join(os.path.dirname(skillsnetwork.__file__), 'core.py')

with open(core_file, 'r') as f:
    content = f.read()

lines = content.split('\n')

print("=== SEARCHING FOR _rmrf FUNCTION ===")
for i, line in enumerate(lines):
    if 'def _rmrf' in line:
        print(f"Found at line {i+1}:")
        # Get the complete function
        start = i
        indent_level = len(line) - len(line.lstrip())
        
        # Find end of function
        end = len(lines)
        for j in range(i+1, len(lines)):
            current_line = lines[j]
            if current_line.strip() and len(current_line) - len(current_line.lstrip()) <= indent_level:
                if current_line.strip().startswith('def ') or current_line.strip().startswith('class '):
                    end = j
                    break
        
        # Print the function
        for k in range(start, end):
            print(f"{k+1:3}: {lines[k]}")
        break

=== SEARCHING FOR _rmrf FUNCTION ===
Found at line 117:
117: def _rmrf(path: Path) -> None:
118:     if path.is_dir():
119:         shutil.rmtree(path)
120:     else:
121:         path.unlink()
122: 
123: 
